In [73]:
import requests
from datetime import datetime, timedelta

# configuração
email = 'jhonatan.cruz@superbet.com'
token = 'd6FQc8anTNI0RHbn0lMvWePn03LKB87uKshEJdqi'
subdomain = 'superbetbr'

# calcular timestamp dos últimos 40 minutos
since_time = datetime.utcnow() - timedelta(minutes=180) - timedelta(minutes=1)  #precisa ser -180 minutos depois a quantidade de tempo 
since_timestamp = int(since_time.timestamp())

# url da API incremental
url = f'https://{subdomain}.zendesk.com/api/v2/incremental/tickets.json?start_time={since_timestamp}&status=open'

# autenticação
auth = (f'{email}/token', token)

# requisição
resp = requests.get(url, auth=auth)
data = resp.json()

# pegar tickets abertos
tickets = data.get('tickets', [])

print(f"Encontrados {len(tickets)} tickets abertos nos últimos 40 minutos")
for t in tickets:
    print(f"ID: {t['id']}, Assunto: {t['subject']}, Atualizado: {t['updated_at']}")


Encontrados 2 tickets abertos nos últimos 40 minutos
ID: 4196803, Assunto: Conversation with TIAGO DA SILVA ARAUJO, Atualizado: 2025-08-29T16:24:28Z
ID: 4195913, Assunto: Conversation with Marcelo, Atualizado: 2025-08-29T16:13:37Z


In [92]:
import requests, duckdb, pandas as pd
from datetime import datetime, timedelta

# Configuração e requisição
auth = ('jhonatan.cruz@superbet.com/token', 'd6FQc8anTNI0RHbn0lMvWePn03LKB87uKshEJdqi')
url = f"https://superbetbr.zendesk.com/api/v2/incremental/tickets.json?start_time={int((datetime.utcnow()-timedelta(minutes=181)).timestamp())}&status=open"
tickets = requests.get(url, auth=auth).json().get('tickets', [])

# DuckDB setup e carga de dados
with duckdb.connect(':memory:') as conn:
    # Convertendo a lista de dicionários para DataFrame
    df = pd.DataFrame(tickets)
    
    # Carregando os dados
    conn.execute("CREATE TABLE zendesk_tickets AS SELECT * FROM df")
    
    # Consultando todas as colunas
    print("\nTotal de tickets:", conn.execute("SELECT COUNT(*) FROM zendesk_tickets").fetchone()[0])
    
    # Exibindo os 5 mais recentes
    print("\nÚltimos 5 tickets:")
    for row in conn.execute("SELECT * FROM zendesk_tickets ORDER BY updated_at DESC LIMIT 5").fetchall():
        print(row)


Total de tickets: 2

Últimos 5 tickets:
('https://superbetbr.zendesk.com/api/v2/tickets/4196825.json', 4196825, None, {'channel': 'native_messaging', 'source': {'from': {}, 'to': {}, 'rel': None}}, '2025-08-29T16:29:14Z', '2025-08-29T16:35:11Z', 1756485398, None, 'Conversation with Fellipe', 'Conversation with Fellipe', 'Conversation with Fellipe', 'normal', 'open', None, 38408121451409, 38408121451409, 32958754793105, None, 18949206362897, [], [], [], None, None, False, False, None, ['agent_escalated', 'bonus__not_granted', 'bot_handled'], [{'id': 18949326292369, 'value': 'bonus__not_granted'}, {'id': 30328564081041, 'value': None}, {'id': 30328585170449, 'value': None}, {'id': 30328596771601, 'value': None}, {'id': 30434665061521, 'value': None}, {'id': 30434724908177, 'value': None}, {'id': 30434785849361, 'value': None}, {'id': 30434957640209, 'value': None}, {'id': 30435069596433, 'value': None}, {'id': 30435129238801, 'value': None}, {'id': 30435140811537, 'value': None}, {'id':

In [150]:
import requests, duckdb, pandas as pd
from datetime import datetime, timedelta

auth = ('jhonatan.cruz@superbet.com/token', 'd6FQc8anTNI0RHbn0lMvWePn03LKB87uKshEJdqi')
start = int((datetime.utcnow() - timedelta(minutes=185)).timestamp())
tickets = requests.get(
    f"https://superbetbr.zendesk.com/api/v2/incremental/tickets.json?start_time={start}&status=open",
    auth=auth
).json().get('tickets', [])

df = pd.DataFrame(tickets)

# extrair canal se existir
if 'via' in df.columns:
    df['via_channel'] = df['via'].apply(lambda x: x.get('channel') if isinstance(x, dict) else None)

# manter apenas colunas existentes
cols = ['id','assignee_id','created_at','updated_at','via_channel']
existing = [c for c in cols if c in df.columns]
if not existing:
    df = pd.DataFrame({cols[0]: []})
else:
    df = df[existing]

with duckdb.connect(':memory:') as conn:
    conn.execute("CREATE TABLE zendesk_tickets AS SELECT * FROM df")
    total = conn.execute("SELECT COUNT(*) FROM zendesk_tickets").fetchone()[0]
    print("\nTotal de tickets:", total)

    # imprimir últimos 5 tickets de forma segura
    if 'updated_at' in df.columns:
        rows = conn.execute("SELECT * FROM zendesk_tickets ORDER BY updated_at DESC LIMIT 5").fetchall()
    else:
        rows = conn.execute("SELECT * FROM zendesk_tickets LIMIT 5").fetchall()
    print("\nÚltimos 5 tickets:")
    for r in rows:
        print(r)



Total de tickets: 85

Últimos 5 tickets:
(4196770, 36502710512145.0, '2025-08-29T16:18:53Z', '2025-08-29T16:42:57Z', 'native_messaging')
(4196200, 33239584419601.0, '2025-08-29T14:45:21Z', '2025-08-29T16:42:53Z', 'native_messaging')
(4196840, 30264600391441.0, '2025-08-29T16:31:20Z', '2025-08-29T16:42:51Z', 'native_messaging')
(4196905, 36152753141137.0, '2025-08-29T16:42:50Z', '2025-08-29T16:42:51Z', 'native_messaging')
(4187643, 32959313480977.0, '2025-08-28T16:13:09Z', '2025-08-29T16:42:49Z', 'side_conversation')


In [234]:
import requests
import duckdb
import pandas as pd
from datetime import datetime, timedelta

# Autenticação
auth = ('jhonatan.cruz@superbet.com/token', 'd6FQc8anTNI0RHbn0lMvWePn03LKB87uKshEJdqi')

# Timestamp inicial (185 minutos atrás)
start = int((datetime.utcnow() - timedelta(minutes=185)).timestamp())

# Buscar tickets
tickets = requests.get(
    f"https://superbetbr.zendesk.com/api/v2/incremental/tickets.json?start_time={start}",
    auth=auth
).json().get('tickets', [])

# Criar DataFrame
df = pd.DataFrame(tickets)

# Função auxiliar para extrair valores de dicionários
def extract_from_dict(col, key):
    return col.apply(lambda x: x.get(key) if isinstance(x, dict) else None)

# Extrair canal se existir
if 'via' in df.columns:
    df['via_channel'] = df['via'].str.get('channel')

# Tratar tags
if 'tags' in df.columns:
    df['tags'] = df['tags'].apply(lambda x: ', '.join(x) if isinstance(x, list) else None)

# Tratar satisfação
if 'satisfaction_rating' in df.columns:
    df['satisfaction_rating'] = pd.to_numeric(df['satisfaction_rating'], errors='coerce')

# Mapear assignee_id para nomes e grupos
if 'assignee_id' in df.columns and not df['assignee_id'].isnull().all():
    user_ids = df['assignee_id'].dropna().unique().astype(int).tolist()
    
    if user_ids:
        # Buscar nomes dos usuários
        users = requests.get(
            f"https://superbetbr.zendesk.com/api/v2/users/show_many.json?ids={','.join(map(str, user_ids))}",
            auth=auth
        ).json().get('users', [])
        id_to_name = {u['id']: u['name'] for u in users}
        df['assignee_name'] = df['assignee_id'].map(id_to_name)

        # Buscar grupos para cada usuário
        id_to_groups = {}
        for user_id in user_ids:
            groups = requests.get(
                f"https://superbetbr.zendesk.com/api/v2/users/{user_id}/groups.json",
                auth=auth
            ).json().get('groups', [])
            id_to_groups[user_id] = ', '.join([g['name'] for g in groups])
        
        df['assignee_groups'] = df['assignee_id'].map(id_to_groups)

# Ajustar timestamps subtraindo 3 horas
for col in ['created_at', 'updated_at']:
    if col in df.columns:
        df[col] = (pd.to_datetime(df[col]) - timedelta(hours=3)).dt.strftime('%Y-%m-%d %H:%M:%S')

# Manter apenas colunas relevantes
cols = [
    'id', 'satisfaction_rating', 'status', 'via_channel', 'assignee_name',
    'type', 'created_at', 'updated_at', 'tags', 'assignee_groups'
]
existing = [c for c in cols if c in df.columns]
df = df[existing] if existing else pd.DataFrame({cols[0]: []})

# Exibir resultados
with duckdb.connect(':memory:') as conn:
    conn.execute("CREATE TABLE zendesk_tickets AS SELECT * FROM df")
    total = conn.execute("SELECT COUNT(*) FROM zendesk_tickets").fetchone()[0]
    print("\nTotal de tickets:", total)

    # Exibir últimos 5 tickets
    if 'updated_at' in df.columns:
        rows = conn.execute("SELECT * FROM zendesk_tickets ORDER BY updated_at DESC LIMIT 5").fetchall()
    else:
        rows = conn.execute("SELECT * FROM zendesk_tickets LIMIT 5").fetchall()

    print("\nÚltimos 5 tickets:")
    for r in rows:
        print(r)



Total de tickets: 67

Últimos 5 tickets:
(4198421, None, 'solved', 'voice', 'Thais Junqueira', 'question', '2025-08-29 17:51:03', '2025-08-29 18:11:23', 'donotclose, error__facematch, ratingreceived', 'CNX SP, Report team - Talk, Support, TL - Nicolas Calipo')
(4198534, None, 'new', 'native_messaging', None, None, '2025-08-29 18:11:17', '2025-08-29 18:11:17', 'agent_escalated, betting__not_settled_yet, bot_handled, sport__offer_info, sport__settlement_info, sport__unsettled_tickets', None)
(4198426, None, 'open', 'native_messaging', 'Tania Toledo', None, '2025-08-29 17:51:57', '2025-08-29 18:11:06', 'account__change_email_request, agent_escalated, bot_handled, customer_didntaccesschat, escalation_validated, has_side_conversation, pending_escalation, pesquisa_escalonamento, reconhecer', 'BKO - TL Lucas Cardoso, CNX SP, Report team - T1 - Live chat, Support')
(4198496, None, 'open', 'native_messaging', 'Tania Toledo', None, '2025-08-29 18:03:39', '2025-08-29 18:11:06', 'agent_escalated,

In [260]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_and_process_zendesk(auth, minutes_back=260):
    """Busca tickets do Zendesk, processa campos e retorna dataframe pronto"""
    start = int((datetime.utcnow() - timedelta(minutes=minutes_back)).timestamp())
    tickets = requests.get(
        f"https://superbetbr.zendesk.com/api/v2/incremental/tickets.json?start_time={start}",
        auth=auth
    ).json().get('tickets', [])
    
    df = pd.DataFrame(tickets)
    
    # Extrair canal se existir
    if 'via' in df.columns:
        df['via_channel'] = df['via'].str.get('channel')
    if 'tags' in df.columns:
        df['tags'] = df['tags'].apply(lambda x: ', '.join(x) if isinstance(x, list) else None)
    if 'satisfaction_rating' in df.columns:
        df['satisfaction_rating'] = pd.to_numeric(df['satisfaction_rating'], errors='coerce')

    # Mapear assignee_id para nomes e grupos
    if 'assignee_id' in df.columns and not df['assignee_id'].isnull().all():
        user_ids = df['assignee_id'].dropna().unique().astype(int).tolist()
        if user_ids:
            users = requests.get(
                f"https://superbetbr.zendesk.com/api/v2/users/show_many.json?ids={','.join(map(str, user_ids))}",
                auth=auth
            ).json().get('users', [])
            id_to_name = {u['id']: u['name'] for u in users}
            df['assignee_name'] = df['assignee_id'].map(id_to_name)
            
            id_to_groups = {}
            for user_id in user_ids:
                groups = requests.get(
                    f"https://superbetbr.zendesk.com/api/v2/users/{user_id}/groups.json",
                    auth=auth
                ).json().get('groups', [])
                id_to_groups[user_id] = ', '.join([g['name'] for g in groups])
            df['assignee_groups'] = df['assignee_id'].map(id_to_groups)
    
    # Ajustar timestamps
    for col in ['created_at', 'updated_at']:
        if col in df.columns:
            df[col] = (pd.to_datetime(df[col]) - timedelta(hours=3)).dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Manter apenas colunas relevantes
    cols = ['id', 'satisfaction_rating','assignee_name', 'created_at','updated_at'
            ,'status','via_channel','type', 'tags','assignee_groups']
    existing = [c for c in cols if c in df.columns]
    df = df[existing] if existing else pd.DataFrame({cols[0]: []})
    
    return df

def show_last_tickets(df, n=20):
    """Função padrão para exibir total de tickets e últimos n tickets com todas as colunas"""
    print(f"\nTotal de tickets: {len(df)}\n")
    
    if 'updated_at' in df.columns:
        last_rows = df.sort_values('updated_at', ascending=False).head(n)
    else:
        last_rows = df.head(n)
    
    for _, row in last_rows.iterrows():
        print(row.to_dict())

# -------------------------
# exemplo de uso:
auth = ('jhonatan.cruz@superbet.com/token', 'd6FQc8anTNI0RHbn0lMvWePn03LKB87uKshEJdqi')
df = fetch_and_process_zendesk(auth)
show_last_tickets(df)



Total de tickets: 1000

{'id': 4198508, 'satisfaction_rating': nan, 'assignee_name': 'Priscila Fagundes', 'created_at': '2025-08-29 18:06:16', 'updated_at': '2025-08-29 21:01:32', 'status': 'closed', 'via_channel': 'native_messaging', 'type': 'question', 'tags': 'agent_escalated, bonus__not_granted, bot_handled, customer_didntaccesschat, escalation_validated, has_side_conversation, pending_escalation, pesquisa_escalonamento, sport__offer_info, sport__settlement_info, sports__bets_error', 'assignee_groups': 'CNX LDR, Report team - T1 - Live chat, Support, T1 - TL Barbara Alves'}
{'id': 4198506, 'satisfaction_rating': nan, 'assignee_name': 'Priscila Fagundes', 'created_at': '2025-08-29 18:05:46', 'updated_at': '2025-08-29 21:01:32', 'status': 'closed', 'via_channel': 'native_messaging', 'type': 'question', 'tags': 'agent_escalated, bonus__eligibility, bot_handled, customer_didntaccesschat, gpt_reply, incident_nocsat, pesq_clientausent', 'assignee_groups': 'CNX LDR, Report team - T1 - Li